In [1]:
import torch
import torchaudio

In [2]:
torchaudio.set_audio_backend("soundfile")

/tmp/ipykernel_55138/1822137616.py:1: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')
import os
# os.chdir('/content/drive/MyDrive/Colab Notebooks/swishnet')
# import sys
# sys.path.append(os.getcwd())

In [4]:
def collect_files(base_dir):
        collected = []
        for sub in os.listdir(base_dir):
            sub_path = os.path.join(base_dir, sub)
            if os.path.isdir(sub_path):
                for fname in os.listdir(sub_path):
                    full_path = os.path.join(sub_path, fname)
                    collected.append(full_path)
        return collected

In [5]:
unpreprocessed_car_dir = "./car"
unpreprocessed_music_dir = "./music"
unpreprocessed_noise_dir = "./noise"
unpreprocessed_speech_dir = "./speech"

In [6]:
unpreprocessed_car_list = collect_files(unpreprocessed_car_dir)
unpreprocessed_music_list = collect_files(unpreprocessed_music_dir)
unpreprocessed_noise_list = collect_files(unpreprocessed_noise_dir)
unpreprocessed_speech_list = collect_files(unpreprocessed_speech_dir)

In [7]:
def get_wav_info_torchaudio(filepath):
    info = torchaudio.info(filepath)
    
    print(f"파일 경로: {filepath}")
    print(f"샘플링 레이트 (Hz): {info.sample_rate}")
    print(f"채널 수: {info.num_channels}")
    print(f"총 프레임 수: {info.num_frames}")
    print(f"재생 시간 (초): {info.num_frames / info.sample_rate:.2f}")
    print(f"비트 깊이: {info.bits_per_sample}")

In [8]:
print(unpreprocessed_car_list[0])
get_wav_info_torchaudio(unpreprocessed_car_list[0])


./car/Cars_2/Street_Sounds____60min__Sleep_Video_181.wav
파일 경로: ./car/Cars_2/Street_Sounds____60min__Sleep_Video_181.wav
샘플링 레이트 (Hz): 44100
채널 수: 2
총 프레임 수: 220500
재생 시간 (초): 5.00
비트 깊이: 16


In [9]:
resampler = torchaudio.transforms.Resample(orig_freq=48000, new_freq=16000)
def preprocess(preprocessed_data_list):
    data_list = []
    for i in range(len(preprocessed_data_list)):
        print(preprocessed_data_list[i])
        waveform, sample_rate = torchaudio.load(preprocessed_data_list[i])
        if sample_rate != 16000:
            waveform = resampler(waveform)
            
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)  # [1, time]
        waveform = waveform.squeeze(0)  # [time]
        cutting_size = 20000
        for i in range(waveform.shape[0]//cutting_size):
            try:
                start = i * cutting_size
                end = (i + 1) * cutting_size
                waveform_segment = waveform[start:end]
                if waveform_segment.shape[0] == cutting_size:
                    data_list.append(waveform_segment)
            except:
                print(f"Error processing segment {i}: {waveform[start:end].shape}")
                continue
    return data_list

In [10]:
car_list = preprocess(unpreprocessed_car_list)
music_list = preprocess(unpreprocessed_music_list)
noise_list = preprocess(unpreprocessed_noise_list)
speech_list = preprocess(unpreprocessed_speech_list)


./car/Cars_2/Street_Sounds____60min__Sleep_Video_181.wav
./car/Cars_2/Street_Sounds____60min__Sleep_Video_179.wav
./car/Cars_2/Street_Sounds____60min__Sleep_Video_177.wav
./car/Cars_2/Street_Sounds____60min__Sleep_Video_176.wav


./car/Cars_2/Street_Sounds____60min__Sleep_Video_178.wav
./car/Cars_2/Street_Sounds____60min__Sleep_Video_180.wav
./car/Cars_2/Street_Sounds____60min__Sleep_Video_186.wav
./car/Cars/Street_Sounds____60min__Sleep_Video_118.wav
./car/Cars/Street_Sounds____60min__Sleep_Video_125.wav
./car/Cars/Street_Sounds____60min__Sleep_Video_102.wav
./car/Cars/Street_Sounds____60min__Sleep_Video_133.wav
./car/Cars/Street_Sounds____60min__Sleep_Video_174.wav
./car/Cars/Street_Sounds____60min__Sleep_Video_110.wav
./car/Cars/Street_Sounds____60min__Sleep_Video_149.wav
./car/Cars/Street_Sounds____60min__Sleep_Video_175.wav
./car/Cars/Street_Sounds____60min__Sleep_Video_064.wav
./car/Cars/Street_Sounds____60min__Sleep_Video_057.wav
./car/Cars/inside_car_wet_driving_SBA_300062047_preview_06.wav
./car/Cars/Street_Sounds____60min__Sleep_Video_025.wav
./car/Cars/Street_Sounds____60min__Sleep_Video_150.wav
./car/Cars/Street_Sounds____60min__Sleep_Video_060.wav
./car/Cars/inside_car_wet_driving_SBA_300062047_pre

In [12]:
print(len(car_list))
print(len(music_list))
print(len(noise_list))
print(len(speech_list))

690
669
2711
669


In [ ]:


torch.save(car_list, "../unsplited/car/car_data.pt")
torch.save(music_list, "../unsplited/music/music_data.pt")
torch.save(noise_list, "../unsplited/noise/noise_data.pt")
torch.save(speech_list, "../unsplited/speech/speech_data.pt")

690
